# Importowanie potrzebnych bibliotek startowych

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Baza Danych

## Import

In [2]:
file_path = 'database/text_and_tags_small.tsv'
dataset = pd.read_csv(file_path, delimiter='\t')

## Oczyszczanie tekstów artykułów

- w pierwszej kolejności usuwamy znaki, niebędące literami oraz zmieniamy wszystkie litery na małe
- kazdy artykuł dzielony jest następnie na liste słów
- przeprowadzamy stemming na słowach, niebędących tzw. "stop words"
- listę ze zmianami łączymy z powrotem w artykuł

Czym jest stemming? 
- jest to proces przekształcania słów do ich formy podstawowej np. "running" do formy "run"

Czym są "stop words"?
- są to słowa często pojawiające się w tekstach, ale nie wnoszą do nich niczego co pomaga w analizie
- np. "the", "is", "at" itd.

In [3]:
import re # importujemy bibliotekę do obsługi wyrażeń regularnych
import nltk # importujemy bibliotekę do przetwarzania języka naturalnego
nltk.download('stopwords') # pobieramy listę "stop words" z biblioteki nltk
from nltk.corpus import stopwords # importujemy "stop words" z nltk.corpus
from nltk.stem.porter import PorterStemmer # importujemy PorterStemmer do stemmingu

corpus = [] # inicjalizujemy pusty korpus
for i in range(0, len(dataset)): # iterujemy przez wszystkie artykuły
    article = re.sub('[^a-zA-Z]', ' ', dataset['text'][i]) # usuwamy wszystkie znaki, które nie są literami
    article = article.lower() # zamieniamy wszystkie litery na małe
    article = article.split() # dzielimy artykuł na listę słów

    ps = PorterStemmer() # inicjalizujemy stemmer
    article = [ps.stem(word) for word in article if not word in set(stopwords.words('english'))] # usuwamy "stop words" i stosujemy stemming
    article = ' '.join(article) # łączymy słowa z powrotem w artykuł
    corpus.append(article) # dodajemy przetworzony artykuł do korpusu

[nltk_data] Downloading package stopwords to /Users/minis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



### Wektoryzacja tekstu

W tym bloku kodu używamy `CountVectorizer` z biblioteki `scikit-learn`, aby przekształcić nasz przetworzony tekst (korpus) w wektory liczbowe, które mogą być użyte do trenowania modeli uczenia maszynowego. `CountVectorizer` konwertuje kolekcję dokumentów tekstowych na macierz liczb całkowitych, reprezentującą liczbę wystąpień słów. Oto co robi każda linia kodu:

- `from sklearn.feature_extraction.text import CountVectorizer`: Importujemy klasę `CountVectorizer`.
- `cv = CountVectorizer(max_features=1500)`: Tworzymy instancję `CountVectorizer`, ograniczając liczbę cech do 1500. Oznacza to, że wektor będzie zawierał 1500 najczęściej występujących słów w korpusie.
- `X = cv.fit_transform(corpus).toarray()`: Dopasowujemy `CountVectorizer` do naszego korpusu i transformujemy tekst na wektory liczbowe. Metoda `toarray()` konwertuje wynik na tablicę numpy, co jest przydatne dla wielu algorytmów uczenia maszynowego, które oczekują danych wejściowych w tej formie.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()

In [ ]:
print(X[:5])

### Przygotowywanie tagów

W tej części kodu wykorzystujemy `MultiLabelBinarizer` z biblioteki `scikit-learn`, aby przekształcić etykiety tekstowe w postaci tagów na binarne wektory. Jest to niezbędne w problemach klasyfikacji wieloetykietowej, gdzie każda próbka może być przypisana do wielu klas jednocześnie. Oto szczegółowy opis działania tego fragmentu:

- `from sklearn.preprocessing import MultiLabelBinarizer`: Importujemy klasę `MultiLabelBinarizer`.
- `mlb = MultiLabelBinarizer()`: Tworzymy instancję `MultiLabelBinarizer`.
- `y = mlb.fit_transform(dataset['tags'])`: Dopasowujemy `MultiLabelBinarizer` do kolumny 'tags' naszego zbioru danych i transformujemy listę tagów do binarnej formy macierzy. Każda kolumna w tej macierzy odpowiada jednemu tagowi, a wartość 1 oznacza, że tag jest przypisany do danej próbki, natomiast 0 oznacza jego brak.

In [ ]:
print(dataset['tags'][:5])

Aktualnie tagi są listą zawierającą pojedyncze elementy, które są ciągami znaków. Musimy przekształcić ten fromat na listę list, gdzie kazda wewnętrzna lista zawiera etykiety jako odzielne ciągi znaków.

In [11]:
import ast

def transform_tags(tag_string):
    try:
        return ast.literal_eval(tag_string)
    except:
        return []

dataset['tags'] = dataset['tags'].apply(transform_tags)
print(dataset['tags'][:2])

0    [Mental Health, Health, Psychology, Science, N...
1    [Mental Health, Coronavirus, Science, Psycholo...
Name: tags, dtype: object


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(dataset['tags'])

In [ ]:
print(mlb.classes_)

In [ ]:
print(y[:3])

### Podział na zbiory testowe i treningowe

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20, random_state=0)

: 

# Nauka modelu

Ten blok kodu jest częścią procesu uczenia maszynowego, w którym tworzony i trenowany jest model klasyfikacji wieloetykietowej za pomocą algorytmu Naive Bayes. Oto szczegółowy opis:

- from sklearn.naive_bayes import MultinomialNB: Importuje klasę MultinomialNB z biblioteki scikit-learn. MultinomialNB to implementacja naiwnego klasyfikatora Bayesa dla rozkładu wielomianowego, który jest odpowiedni dla danych z cechami dyskretnymi (np. liczba wystąpień słowa w tekście).

- from sklearn.multioutput import MultiOutputClassifier: Importuje klasę MultiOutputClassifier, która jest strategią do rozszerzenia klasyfikatorów jednoetykietowych na klasyfikatory wieloetykietowe. Pozwala to na przewidywanie wielu zależnych zmiennych kategorialnych (etykiet).

- model = MultiOutputClassifier(MultinomialNB()): Tworzy instancję MultiOutputClassifier, przekazując do niej instancję MultinomialNB jako estymator bazowy. To oznacza, że dla każdej etykiety (kolumny w y_train) zostanie utworzony osobny klasyfikator MultinomialNB.

- model.fit(X_train, y_train): Metoda fit jest używana do trenowania modelu na podstawie danych treningowych. X_train zawiera wektory cech (przetworzone teksty), a y_train zawiera odpowiadające im etykiety w formie binarnej. Model uczy się, jak przewidywać etykiety na podstawie cech wejściowych.

Po wykonaniu tego kodu, model jest wytrenowanym klasyfikatorem wieloetykietowym, który może być używany do przewidywania etykiet dla nowych, nieznanych próbek.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier

# Import klasy MultinomialNB z biblioteki scikit-learn
# MultinomialNB to implementacja naiwnego klasyfikatora Bayesa dla rozkładu wielomianowego
model = MultiOutputClassifier(MultinomialNB())

# Utworzenie modelu klasyfikacji wieloetykietowej
# MultiOutputClassifier pozwala na rozszerzenie klasyfikatorów jednoetykietowych
# na klasyfikatory wieloetykietowe
model.fit(X_train, y_train)  # Trenowanie modelu na podstawie danych treningowych


Ten blok kodu służy do przewidywania etykiet dla zestawu testowego (`X_test`) przy użyciu wcześniej wytrenowanego modelu klasyfikacji wieloetykietowej. Wywołanie metody `predict` na obiekcie `model` powoduje, że model stosuje nauczony algorytm do nowych danych i generuje przewidywane etykiety (`y_pred`).

- `y_pred = model.predict(X_test)`: Przewiduje etykiety dla danych testowych `X_test` wykorzystując wytrenowany model. Wynik przypisany do zmiennej `y_pred` zawiera przewidziane etykiety w formie binarnej, gdzie każda kolumna odpowiada jednej etykiecie, a wartość 1 oznacza przewidzienie obecności etykiety, a 0 jej brak.

In [ ]:
y_pred = model.predict(X_test)

# Testowanie modelu

### Hamming Loss
-  **Opis:** Hamming Loss to miara błędu używana w klasyfikacji wieloetykietowej. Określa średnią liczbę etykiet, które są źle przewidywane, tj. niewłaściwa etykieta jest przewidywana jako obecna lub właściwa etykieta jest przewidywana jako nieobecna.
-  **Sposób interpretacji:** Im niższa wartość Hamming Loss, tym lepszy jest model. Wartość 0 oznacza, że nie ma błędów w przewidywaniu etykiet. Wartość bliższa 1 wskazuje na większą liczbę błędów. W kontekście klasyfikacji wieloetykietowej, nawet niewielkie wartości Hamming Loss mogą być uważane za dobre wyniki, biorąc pod uwagę złożoność zadania.

### Precyzja
-  **Opis:** Precyzja (precision) to miara, która określa, jaki procent etykiet przewidzianych jako pozytywne przez model jest faktycznie pozytywny. Innymi słowy, jest to stosunek prawidłowo przewidzianych pozytywnych instancji do ogólnej liczby instancji przewidzianych jako pozytywne.
-  **Sposób interpretacji:** Wysoka precyzja oznacza, że model rzadko oznacza negatywną etykietę jako pozytywną. Niska precyzja wskazuje, że model często błędnie klasyfikuje negatywne etykiety jako pozytywne. W idealnym przypadku precyzja powinna być jak najbliższa 1.

### Pełność (Recall)
-  **Opis:** Pełność (recall) to miara zdolności modelu do znalezienia wszystkich pozytywnych instancji. Określa, jaki procent rzeczywistych pozytywnych etykiet został poprawnie zidentyfikowany przez model.
-  **Sposób interpretacji:** Wysoka pełność oznacza, że model dobrze radzi sobie z identyfikacją pozytywnych etykiet, nawet kosztem zwiększenia liczby fałszywie pozytywnych przewidywań. Niska pełność wskazuje, że model przegapił wiele pozytywnych etykiet. Idealna pełność to wartość 1, co oznacza, że wszystkie pozytywne etykiety zostały poprawnie przewidziane.

In [ ]:
from sklearn.metrics import precision_score, recall_score, hamming_loss

# Hamming Loss
hamming = hamming_loss(y_test, y_pred)

# Oblicz precyzję
precision = precision_score(y_test, y_pred, average='micro')

# Oblicz pełność (recall)
recall = recall_score(y_test, y_pred, average='micro')

print("Hamming Loss: ", hamming)
print("Precyzja: ", precision)
print("Pełność: ", recall)

---

# Eksport gotowego modelu

In [ ]:
from joblib import dump
dump(model, 'model_100k/model.joblib')
dump(cv, 'model_100k/count_vectorizer.joblib')
dump(mlb, 'model_100k/multilabel_binarizer.joblib')